# Merging for Granted Patents

start by importing packages and setting up some settings

In [1]:
#import packages
import pandas as pd
import numpy as np


#see all columns and up to 150 rows as needed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

# Load in the patentsview.org combined dataset

In [2]:
#load in the data
patents = pd.read_csv('reduced_2009_2019.csv')

/Applications/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
patents.patent_number = patents.patent_number.astype('str')
patents.dtypes


Unnamed: 0             int64
patent_number         object
assignee              object
grant_year             int64
application_year       int64
application_number     int64
country               object
city                  object
state                 object
county                object
ipc_section           object
team_size              int64
inventors              int64
men_inventors          int64
women_inventors        int64
dtype: object

In [4]:
#drop the old index column
patents = patents.drop('Unnamed: 0', axis =1 )

In [5]:
#check out that it worked 
patents

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors
0,7472428,"NIKE, INC.",2009,2004,10814594,US,Beaverton,OR,Washington,A,2,2,2,0
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,Vista,CA,San Diego,E,4,4,4,0
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,Vista,CA,San Diego,A,1,1,1,0
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,Indianapolis,IN,Marion,E F G,2,2,2,0
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,Travelers Rest,SC,Greenville,E,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533890,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,Canby,OR,Clackamas,NaN,2,2,2,0
1533891,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,Perry,OK,Noble,NaN,2,2,2,0
1533892,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,Lake Forest,CA,Orange,G,4,4,4,0
1533893,RE47664,APPLE INC.,2019,2017,29622909,US,Cupertino,CA,Santa Clara,NaN,14,14,14,0


In [6]:
#a bunch of them don't have county so can't merge on county and state for fips
#patents.query("county.isnull()", engine = 'python')

In [7]:
# looking at app year
#patents.application_year.sort_values() #.value_counts()

In [8]:
# earliest application year 1969 for granted bw 2009 -2019 
#patents.sort_values(by ='application_year')

## don't have GEOID in the data need it


In [6]:
patents.info()
# lots missing county and some missing city 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1533895 entries, 0 to 1533894
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   patent_number       1533895 non-null  object
 1   assignee            1533895 non-null  object
 2   grant_year          1533895 non-null  int64 
 3   application_year    1533895 non-null  int64 
 4   application_number  1533895 non-null  int64 
 5   country             1533895 non-null  object
 6   city                1532744 non-null  object
 7   state               1533850 non-null  object
 8   county              1433937 non-null  object
 9   ipc_section         1412685 non-null  object
 10  team_size           1533895 non-null  int64 
 11  inventors           1533895 non-null  int64 
 12  men_inventors       1533895 non-null  int64 
 13  women_inventors     1533895 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 163.8+ MB


In [7]:
#about 100,000 don't have counties -> so need to merge on something else
sum(patents.county.isna())

99958

In [8]:
#with out city and county -> can not get fips -> dropping them 
patents.query("city.isnull() & county.isnull()", engine = 'python')

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors
1458,7475569,"THE BOC GRPOUP, INC.",2009,2003,10712904,US,NaN,NJ,NaN,C,3,3,3,0
8403,7490885,"NISSAN TECHNICAL CENTER NORTH AMERICA, INC.",2009,2008,12037624,US,NaN,MI,NaN,B,1,1,0,1
8439,7491002,"CRITERION MANUFACTURING SOLUTIONS, INC.",2009,2005,11088736,US,NaN,GA,NaN,G,3,3,3,0
9992,7494403,EDGECRAFT CORPORATION,2009,2007,11676597,US,NaN,PA,NaN,B,1,1,1,0
11784,7498279,"OWENS CORNING INTELLECTUAL CAPITAL, LLC",2009,2007,11725344,US,NaN,DE,NaN,B,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518517,10416495,HISENSE USA CORPORATION,2019,2017,15853598,US,NaN,GA,NaN,G,1,1,1,0
1520553,D839809,"TURN 5, INC.",2019,2017,29602013,US,NaN,PA,NaN,NaN,1,1,1,0
1525550,D851425,"KEEZIO GROUP, LLC",2019,2017,29623261,US,NaN,CA,NaN,NaN,1,1,1,0
1530280,D862502,LEARNING CIRCLE KIDS LLC,2019,2017,29591207,US,NaN,WA,NaN,NaN,3,3,0,3


# cant group by county if don't know city, county and fips not in data yet

In [9]:
patents = patents.query("city.notnull() | county.notnull()", engine = 'python')

In [10]:
#upper case to besure of any matching issues
patents = patents.assign(city = patents.city.str.strip().str.upper(), 
                         county = patents.county.str.strip().str.upper())

patents

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,WASHINGTON,A,2,2,2,0
1,7472430,WATKINS MANUFACTURING CORPORATION,2009,2006,11604596,US,VISTA,CA,SAN DIEGO,E,4,4,4,0
2,7472431,WATKINS MANUFACTURING CORPORATION,2009,2004,10899547,US,VISTA,CA,SAN DIEGO,A,1,1,1,0
3,7472433,MASCO CORPORATION OF INDIANA,2009,2006,11325927,US,INDIANAPOLIS,IN,MARION,E F G,2,2,2,0
4,7472434,"T&S BRASS AND BRONZE WORKS, INC.",2009,2008,12052249,US,TRAVELERS REST,SC,GREENVILLE,E,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533890,RE47549,"S.R. SMITH, LLC",2019,2015,29515597,US,CANBY,OR,CLACKAMAS,NaN,2,2,2,0
1533891,RE47572,"THE CHARLES MACHINE WORKS, INC.",2019,2018,29655614,US,PERRY,OK,NOBLE,NaN,2,2,2,0
1533892,RE47657,"TOSHIBA AMERICA BUSINESS SOLUTIONS, INC.",2019,2017,15845923,US,LAKE FOREST,CA,ORANGE,G,4,4,4,0
1533893,RE47664,APPLE INC.,2019,2017,29622909,US,CUPERTINO,CA,SANTA CLARA,NaN,14,14,14,0


## split patents into those with county and those without

- those with county 
    - get geoid b y merging on county / state
- those without county, but with city
    - get geoid by merging on city/state

In [11]:
patents.county.isnull().sum()

98807

In [12]:
patents_with_county = patents.query("county.notnull()", engine = 'python')
patents_NO_county = patents.query("county.isnull()", engine = 'python')

{patents_with_county.shape, patents_NO_county.shape}

{(98807, 14), (1433937, 14)}

In [16]:
#patents.query("city == 'FARMINGTON'").county.unique()

In [17]:
#patents.query("city == 'FARMINGTON'").state.unique()

# Load in crosswalks to start to merge

- crosswalk only for city,state 
- for places without county trying to get the geoid so can get county later 
    - see below
    
    

In [13]:
import os
os.chdir('/Users/chewy2.0/Capstone/data/pregrant')

city_crosswalk2 = pd.read_csv("location_crosswalk.csv", dtype = {'GEOID' : 'str'}, low_memory = False)
city_crosswalk2

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
0,f9139cb2-cb8f-11eb-9615-121df0c29c1e,06075,San Francisco,NaN,CA,NaN,US,37.7292,NaN,-123.0470,NaN,San Francisco,NaN,6.0,6075.0,NaN
1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e,32031,Reno,NaN,NV,NaN,US,39.5504,NaN,-119.8030,NaN,Washoe,NaN,32.0,32031.0,NaN
2,9596257a-cb90-11eb-9615-121df0c29c1e,50021,Rutland,NaN,VT,NaN,US,43.6106,NaN,-72.9726,NaN,Rutland,NaN,50.0,50021.0,NaN
3,499f8deb-cb8e-11eb-9615-121df0c29c1e,34003,Parkridge,Park Ridge,NJ,NJ,US,NaN,-74.039390,NaN,41.033790,NaN,Bergen,34.0,NaN,34003.0
4,ea59a211-09be-11ec-893a-12de62d610b1,06085,Sunnyvale,NaN,CA,NaN,US,37.3688,NaN,-122.0360,NaN,Santa Clara,NaN,6.0,6085.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40947,5ccdf65e-cb8e-11eb-9615-121df0c29c1e,18011,Zlonsville,Zionsville,IN,IN,US,NaN,-86.262240,NaN,39.950460,NaN,Boone,18.0,NaN,18011.0
40948,8004bb30-cb8e-11eb-9615-121df0c29c1e,29031,Santa C,Cape Girardeau,CA,MO,US,41.6644,-89.524794,-120.7640,37.315383,NaN,Cape Girardeau,6.0,NaN,29031.0
40949,68951f3a-cb8f-11eb-9615-121df0c29c1e,06013,Kinsington,West Contra Costa,CA,CA,US,NaN,-122.281350,NaN,37.912850,NaN,Contra Costa,6.0,NaN,6013.0
40950,a0cb02ea-cbae-11eb-9615-121df0c29c1e,42111,Tire Hill,Conemaugh,PA,PA,US,40.2688,-78.915490,-78.9160,40.268520,NaN,Somerset,42.0,NaN,42111.0


In [14]:
#change directory back 
os.chdir('/Users/chewy2.0/Capstone')

In [16]:
#city_crosswalk = pd.read_csv('crosswalk_city_state_name_to_city_state_fips.csv')




#upper case to be sure match
#city_crosswalk.city = city_crosswalk.city.str.strip().str.upper()
city_crosswalk2.pv_city = city_crosswalk2.pv_city.str.strip().str.upper()
city_crosswalk2.pv_county = city_crosswalk2.pv_county.str.strip().str.upper()
#city_crosswalk.head()

city_crosswalk2.head()

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
0,f9139cb2-cb8f-11eb-9615-121df0c29c1e,06075,SAN FRANCISCO,NaN,CA,NaN,US,37.7292,NaN,-123.0470,NaN,SAN FRANCISCO,NaN,6.0,6075.0,NaN
1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e,32031,RENO,NaN,NV,NaN,US,39.5504,NaN,-119.8030,NaN,WASHOE,NaN,32.0,32031.0,NaN
2,9596257a-cb90-11eb-9615-121df0c29c1e,50021,RUTLAND,NaN,VT,NaN,US,43.6106,NaN,-72.9726,NaN,RUTLAND,NaN,50.0,50021.0,NaN
3,499f8deb-cb8e-11eb-9615-121df0c29c1e,34003,PARKRIDGE,Park Ridge,NJ,NJ,US,NaN,-74.03939,NaN,41.03379,NaN,Bergen,34.0,NaN,34003.0
4,ea59a211-09be-11ec-893a-12de62d610b1,06085,SUNNYVALE,NaN,CA,NaN,US,37.3688,NaN,-122.0360,NaN,SANTA CLARA,NaN,6.0,6085.0,NaN


In [21]:
#city_crosswalk2.shape

In [22]:
#city_crosswalk2.query("pv_city == 'SAN FRANCISCO'")

## pv_city -> this is patent view data

- since it's trash => replaced jacked up ones with gl_state
- merge two columns into corrected state? 

In [23]:
city_crosswalk2.query("gl_state != pv_state")
#city_crosswalk2.query("gl_city.isnull()", engine = 'python')

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips
0,f9139cb2-cb8f-11eb-9615-121df0c29c1e,06075,SAN FRANCISCO,NaN,CA,NaN,US,37.7292,NaN,-123.0470,NaN,SAN FRANCISCO,NaN,6.0,6075.0,NaN
1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e,32031,RENO,NaN,NV,NaN,US,39.5504,NaN,-119.8030,NaN,WASHOE,NaN,32.0,32031.0,NaN
2,9596257a-cb90-11eb-9615-121df0c29c1e,50021,RUTLAND,NaN,VT,NaN,US,43.6106,NaN,-72.9726,NaN,RUTLAND,NaN,50.0,50021.0,NaN
4,ea59a211-09be-11ec-893a-12de62d610b1,06085,SUNNYVALE,NaN,CA,NaN,US,37.3688,NaN,-122.0360,NaN,SANTA CLARA,NaN,6.0,6085.0,NaN
5,e0172947-09bd-11ec-893a-12de62d610b1,53033,REDMOND,NaN,WA,NaN,US,47.6697,NaN,-122.1230,NaN,KING,NaN,53.0,53033.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40942,5c6c7f5d-cb8e-11eb-9615-121df0c29c1e,41043,WILLAMETTE,Albany,IL,OR,US,NaN,-123.082660,NaN,44.638550,NaN,Linn,17.0,NaN,41043.0
40945,cd8a3f1a-cb90-11eb-9615-121df0c29c1e,39169,MOUNT EATON,NaN,OH,NaN,US,40.6945,NaN,-81.7032,NaN,WAYNE,NaN,39.0,39169.0,NaN
40946,5316e2b3-cbae-11eb-9615-121df0c29c1e,01087,SHORTER,NaN,AL,NaN,US,32.3966,NaN,-85.9345,NaN,MACON,NaN,1.0,1087.0,NaN
40948,8004bb30-cb8e-11eb-9615-121df0c29c1e,29031,SANTA C,Cape Girardeau,CA,MO,US,41.6644,-89.524794,-120.7640,37.315383,NaN,Cape Girardeau,6.0,NaN,29031.0


In [17]:
# fixed state column not in state!
city_crosswalk2 = city_crosswalk2.assign(state = city_crosswalk2['gl_state'])

for i in range(len(city_crosswalk2)):
    if pd.isnull(city_crosswalk2.loc[i,'state']):
        city_crosswalk2.loc[i,'state'] = city_crosswalk2.loc[i,'pv_state']
    else:
        pass
    
city_crosswalk2

,id,GEOID,pv_city,gl_city,pv_state,gl_state,country,pv_lat,gl_lat,pv_long,gl_long,pv_county,gl_county,pv_state_fips,pv_county_fips,gl_county_fips,state
0,f9139cb2-cb8f-11eb-9615-121df0c29c1e,06075,SAN FRANCISCO,NaN,CA,NaN,US,37.7292,NaN,-123.0470,NaN,SAN FRANCISCO,NaN,6.0,6075.0,NaN,CA
1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e,32031,RENO,NaN,NV,NaN,US,39.5504,NaN,-119.8030,NaN,WASHOE,NaN,32.0,32031.0,NaN,NV
2,9596257a-cb90-11eb-9615-121df0c29c1e,50021,RUTLAND,NaN,VT,NaN,US,43.6106,NaN,-72.9726,NaN,RUTLAND,NaN,50.0,50021.0,NaN,VT
3,499f8deb-cb8e-11eb-9615-121df0c29c1e,34003,PARKRIDGE,Park Ridge,NJ,NJ,US,NaN,-74.039390,NaN,41.033790,NaN,Bergen,34.0,NaN,34003.0,NJ
4,ea59a211-09be-11ec-893a-12de62d610b1,06085,SUNNYVALE,NaN,CA,NaN,US,37.3688,NaN,-122.0360,NaN,SANTA CLARA,NaN,6.0,6085.0,NaN,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40947,5ccdf65e-cb8e-11eb-9615-121df0c29c1e,18011,ZLONSVILLE,Zionsville,IN,IN,US,NaN,-86.262240,NaN,39.950460,NaN,Boone,18.0,NaN,18011.0,IN
40948,8004bb30-cb8e-11eb-9615-121df0c29c1e,29031,SANTA C,Cape Girardeau,CA,MO,US,41.6644,-89.524794,-120.7640,37.315383,NaN,Cape Girardeau,6.0,NaN,29031.0,MO
40949,68951f3a-cb8f-11eb-9615-121df0c29c1e,06013,KINSINGTON,West Contra Costa,CA,CA,US,NaN,-122.281350,NaN,37.912850,NaN,Contra Costa,6.0,NaN,6013.0,CA
40950,a0cb02ea-cbae-11eb-9615-121df0c29c1e,42111,TIRE HILL,Conemaugh,PA,PA,US,40.2688,-78.915490,-78.9160,40.268520,NaN,Somerset,42.0,NaN,42111.0,PA


In [25]:
#city_crosswalk2.query("pv_county.isnull() & GEOID.isnull()", engine = 'python')

In [18]:
#city_crosswalk2.groupby(['GEOID'], as_index = False).size().sort_values(by = 'size')
city_crosswalk3 = city_crosswalk2.groupby(['pv_city','state','GEOID'], as_index = False).size().sort_values(by = 'size')
#city_crosswalk3 = city_crosswalk2.groupby(['pv_county', 'pv_city', 'pv_state','GEOID'], as_index = False).size().sort_values(by = 'size')
city_crosswalk3

,pv_city,state,GEOID,size
0,.,MA,25027,1
26546,OOLTEWAH,TN,47065,1
26547,OOLTEWAN,TN,47065,1
26548,OOLTEWATH,TN,47065,1
26549,OOLTEWAY,TN,47065,1
...,...,...,...,...
6871,CHICAGO,IL,17031,5
1591,ATLANTA,GA,13121,5
8335,CORVALLIS,OR,41003,5
31431,SAN DIEGO,CA,06073,6


# issue with the cross walk -> fix and rejoin 

In [19]:
#this is the issue
# all farmingtons have same GEOID
#city_crosswalk.query("city == 'FARMINGTON'")


#this is fixed
city_crosswalk3.query("pv_city == 'FARMINGTON'")

,pv_city,state,GEOID,size
12009,FARMINGTON,UT,49011,1
12010,FARMINGTON,WA,53075,1
12008,FARMINGTON,PA,42051,1
12007,FARMINGTON,NY,36069,1
12004,FARMINGTON,MO,29187,1
11996,FARMINGTON,AR,05143,1
11997,FARMINGTON,CT,09003,1
11998,FARMINGTON,IA,19177,1
11999,FARMINGTON,IL,17057,1
12000,FARMINGTON,KY,21083,1


In [28]:
#city_crosswalk3.duplicated().sum()

In [29]:
#city_crosswalk3.query(" GEOID == '51770'")

In [20]:
# duplicates
city_crosswalk3[city_crosswalk3[['pv_city', 'state']].duplicated()]

,pv_city,state,GEOID,size
26752,OSCEOLA,IA,19143,1
28685,POINCIANA,FL,12105,1
28639,PLYMOUTH,IA,19149,1
28067,PICKFORD,MI,26097,1
21589,MAPLE GLEN,PA,42125,1
22640,MERIDIEN,CA,06051,1
38348,WESTFIELD,WI,55111,1
30048,RICHARD,CA,06087,1
6907,CHILE,NY,36061,1
5392,CAMBRIDGE,WI,55025,1


In [21]:
city_crosswalk3.drop(26752, inplace = True)
city_crosswalk3.query("pv_city == 'OSCEOLA'").sort_values(by = 'state')

,pv_city,state,GEOID,size
26749,OSCEOLA,AR,05093,1
26750,OSCEOLA,FL,12057,1
26751,OSCEOLA,IA,19039,1
26753,OSCEOLA,IN,18141,1
26754,OSCEOLA,MO,29185,1
26755,OSCEOLA,WI,55095,1


In [22]:
city_crosswalk3.drop(28685, inplace = True)
city_crosswalk3.query("pv_city == 'POINCIANA'").sort_values(by = 'state')

,pv_city,state,GEOID,size
28684,POINCIANA,FL,12097,1


In [23]:
city_crosswalk3.drop(28639, inplace = True)
city_crosswalk3.query("pv_city == 'PLYMOUTH'").sort_values(by = 'state')

,pv_city,state,GEOID,size
28635,PLYMOUTH,CA,06005,1
28636,PLYMOUTH,CT,09005,1
28637,PLYMOUTH,FL,12095,1
28638,PLYMOUTH,IA,19033,1
28640,PLYMOUTH,IL,17067,1
28641,PLYMOUTH,IN,18099,1
28642,PLYMOUTH,MA,25023,1
28643,PLYMOUTH,MI,26163,1
28644,PLYMOUTH,MN,27053,2
28645,PLYMOUTH,NC,37187,1


In [24]:
city_crosswalk3.drop(28067, inplace = True)
city_crosswalk3.query("pv_city == 'PICKFORD'").sort_values(by = 'state')

,pv_city,state,GEOID,size
28066,PICKFORD,MI,26033,1


In [25]:
city_crosswalk3.drop(21589, inplace = True)
city_crosswalk3.query("pv_city == 'MAPLE GLEN'").sort_values(by = 'state')

,pv_city,state,GEOID,size
21588,MAPLE GLEN,PA,42091,1


In [26]:
city_crosswalk3.drop(22640, inplace = True)
city_crosswalk3.query("pv_city == 'MERIDIEN'").sort_values(by = 'state')

,pv_city,state,GEOID,size
22639,MERIDIEN,CA,06037,1
22641,MERIDIEN,ID,16001,1


In [27]:
city_crosswalk3.drop(38348	, inplace = True)
city_crosswalk3.query("pv_city == 'WESTFIELD'").sort_values(by = 'state')

,pv_city,state,GEOID,size
38338,WESTFIELD,IA,19149,1
38339,WESTFIELD,IL,17023,1
38340,WESTFIELD,IN,18057,1
38341,WESTFIELD,MA,25013,2
38342,WESTFIELD,NC,37171,1
38343,WESTFIELD,NJ,34039,1
38344,WESTFIELD,NY,36013,1
38345,WESTFIELD,OH,39117,1
38346,WESTFIELD,PA,42117,1
38347,WESTFIELD,WI,55077,1


In [28]:
city_crosswalk3.drop(30047, inplace = True)
city_crosswalk3.query("pv_city == 'RICHARD'").sort_values(by = 'state')

,pv_city,state,GEOID,size
30048,RICHARD,CA,06087,1
30049,RICHARD,LA,22101,2


In [29]:
city_crosswalk3.drop(6907, inplace = True)
city_crosswalk3.query("pv_city == 'CHILE'").sort_values(by = 'state')

,pv_city,state,GEOID,size
6906,CHILE,NY,36055,1


In [30]:
city_crosswalk3.drop(5392, inplace = True)
city_crosswalk3.query("pv_city == 'CAMBRIDGE'").sort_values(by = 'state')

,pv_city,state,GEOID,size
5378,CAMBRIDGE,IA,19169,1
5379,CAMBRIDGE,ID,16087,1
5380,CAMBRIDGE,IL,17073,1
5381,CAMBRIDGE,MA,25017,5
5382,CAMBRIDGE,MD,24019,1
5383,CAMBRIDGE,MN,27059,1
5384,CAMBRIDGE,NE,31065,1
5385,CAMBRIDGE,NJ,34005,1
5386,CAMBRIDGE,NY,36115,1
5387,CAMBRIDGE,OH,39059,1


In [31]:
city_crosswalk3.drop(16074, inplace = True)
city_crosswalk3.query("pv_city == 'HILLSBORO'").sort_values(by = 'state')

,pv_city,state,GEOID,size
16075,HILLSBORO,CA,06099,1
16076,HILLSBORO,CO,08123,1
16077,HILLSBORO,IA,19087,1
16078,HILLSBORO,IL,17135,1
16079,HILLSBORO,IN,18045,1
16080,HILLSBORO,KS,20115,1
16081,HILLSBORO,MD,24011,1
16082,HILLSBORO,MO,29099,1
16083,HILLSBORO,ND,38097,1
16084,HILLSBORO,NH,33011,1


In [32]:
city_crosswalk3.drop(16676, inplace = True)
city_crosswalk3.query("pv_city == 'HOUSTON'").sort_values(by = 'state')

,pv_city,state,GEOID,size
16669,HOUSTON,CA,06089,1
16670,HOUSTON,LA,22003,1
16671,HOUSTON,MN,27055,1
16672,HOUSTON,MO,29215,1
16673,HOUSTON,MS,28017,1
16674,HOUSTON,OH,39149,1
16675,HOUSTON,PA,42125,1
16677,HOUSTON,TX,48201,1
16678,HOUSTON,WI,55109,1


In [33]:
city_crosswalk3.drop(2061, inplace = True)
city_crosswalk3.query("pv_city == 'BALTIMORE'").sort_values(by = 'state')

,pv_city,state,GEOID,size
2058,BALTIMORE,CA,06061,1
2059,BALTIMORE,DE,10003,1
2060,BALTIMORE,MD,24005,1
2062,BALTIMORE,NC,37197,1
2063,BALTIMORE,NY,36023,1
2064,BALTIMORE,OH,39045,1
2065,BALTIMORE,VT,50027,1


In [34]:
city_crosswalk3.drop(15148, inplace = True)
city_crosswalk3.query("pv_city == 'HANOVER'").sort_values(by = 'state')

,pv_city,state,GEOID,size
15137,HANOVER,IA,19021,1
15138,HANOVER,IL,17085,1
15139,HANOVER,IN,18077,1
15140,HANOVER,KS,20201,1
15141,HANOVER,MA,25023,1
15142,HANOVER,MD,24027,1
15143,HANOVER,ME,23017,1
15144,HANOVER,MI,26075,1
15145,HANOVER,MN,27171,1
15146,HANOVER,NH,33009,1


In [35]:
city_crosswalk3.drop(12759, inplace = True)
city_crosswalk3.query("pv_city == 'FRANKLIN'").sort_values(by = 'state')

,pv_city,state,GEOID,size
12741,FRANKLIN,FL,12037,1
12742,FRANKLIN,GA,13149,1
12743,FRANKLIN,ID,16041,1
12744,FRANKLIN,IL,17137,1
12745,FRANKLIN,IN,18081,1
12746,FRANKLIN,KY,21213,1
12747,FRANKLIN,LA,22101,1
12748,FRANKLIN,MA,25021,1
12749,FRANKLIN,MI,26125,1
12750,FRANKLIN,MN,27129,1


In [36]:
city_crosswalk3.query("pv_city == 'SANTA CLARA'")

,pv_city,state,GEOID,size
31595,SANTA CLARA,CA,06037,1
31597,SANTA CLARA,FL,12039,1
31598,SANTA CLARA,OR,41039,1
31599,SANTA CLARA,TX,48363,1
31600,SANTA CLARA,UT,49053,1
31596,SANTA CLARA,CA,06085,4


In [37]:
#city_crosswalk3 = city_crosswalk3[~city_crosswalk3.query("pv_city == 'SANTA CLARA' & state == 'CA' & GEOID == '06037'")]
#city_crosswalk3.query("pv_city == 'SANTA CLARA' & state == 'CA' & GEOID == '06037'")
city_crosswalk3.drop(31595, inplace = True)
city_crosswalk3.query("pv_city == 'SANTA CLARA' & state == 'CA' & GEOID == '06037'")
#removed error

,pv_city,state,GEOID,size


In [48]:
#correct now
#city_crosswalk3.query("pv_city == 'SAN DIEGO'")

In [49]:
#correct now
#city_crosswalk3.query("pv_city == 'THORTON'")

In [38]:
city_crosswalk3.GEOID.isnull().sum()

0

In [39]:
#got rid of sanfran - for geoid
city_crosswalk3 = city_crosswalk3.query(" GEOID != '_'")
#city_crosswalk3 = city_crosswalk3.assign(length_geoid = city_crosswalk3.GEOID.str.len)
city_crosswalk3.query("pv_city == 'SAN FRANCISCO'")

,pv_city,state,GEOID,size
31440,SAN FRANCISCO,AZ,04019,1
31441,SAN FRANCISCO,CA,06075,7


In [40]:
## duplicated fixed?

city_crosswalk3[city_crosswalk3[['pv_city', 'state']].duplicated()]

,pv_city,state,GEOID,size


In [53]:
#good now
#city_crosswalk3.query("pv_city == 'ROME'")

In [54]:
#city_crosswalk3.query("pv_city == 'FALSTON'")

In [55]:
#city_crosswalk3.query("pv_city == 'GULIFORD'")

In [56]:
## the merge doesnt work -> many to many? 

In [57]:
#test it out
# test_merge = pd.merge(patents_NO_county, city_crosswalk3,
#                        left_on = ['city', 'state'],
#                        right_on = ['pv_city', 'state'],
#                        how = 'outer',
#                        indicator = 'matched',
#                        validate = 'many_to_one')

# #not all match becasue crosswalk only made for places without a county
# test_merge.query("matched == 'left_only'")

In [58]:
#pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['city', 'state']).size()).sort_values(by='state')

In [41]:
# merge for real with no county

patents_NO_county_geoid = pd.merge(patents_NO_county, city_crosswalk3,
                        left_on = ['city', 'state'],
                        right_on = ['pv_city', 'state'],
                        how = 'inner')
patents_NO_county_geoid

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,pv_city,GEOID,size
0,7472531,"FLEECEWORKS, INC.",2009,2005,11218805,US,CITY OF INDUSTRY,CA,NaN,B,1,1,0,1,CITY OF INDUSTRY,06037,1
1,7487807,"LINCOLN GLOBAL, INC.",2009,2006,11497711,US,CITY OF INDUSTRY,CA,NaN,B,2,2,2,0,CITY OF INDUSTRY,06037,1
2,7491910,"LINCOLN GLOBAL, INC.",2009,2006,11387025,US,CITY OF INDUSTRY,CA,NaN,B,3,3,2,1,CITY OF INDUSTRY,06037,1
3,7495193,"LINCOLN GLOBAL, INC.",2009,2005,11081059,US,CITY OF INDUSTRY,CA,NaN,B,3,3,3,0,CITY OF INDUSTRY,06037,1
4,7495766,"LINCCLN GLOBAL, INC.",2009,2006,11472696,US,CITY OF INDUSTRY,CA,NaN,G,2,2,2,0,CITY OF INDUSTRY,06037,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96599,D865380,"NOBLE HOUSE HOME FURNISHING, LLC",2019,2017,29621633,US,CHATS WORTH,CA,NaN,NaN,2,2,2,0,CHATS WORTH,06037,1
96600,D867125,"INNO-PAK, LLC",2019,2017,29604217,US,DELWARE,OH,NaN,NaN,2,2,2,0,DELWARE,39041,1
96601,D869289,"INNO-PAK, LLC",2019,2018,29666033,US,DELWARE,OH,NaN,NaN,2,2,2,0,DELWARE,39041,1
96602,D867753,"GIFFIN, INC.",2019,2018,29659105,US,LUMMI ISLAND,WA,NaN,NaN,1,1,1,0,LUMMI ISLAND,53073,1


## Most of the patents don't get a GEOID -> just for places with no county

- separated them so all no county now have geoid 
- need to merge on county and state for rest

In [60]:
#now do it for real 

# patents_merged = pd.merge(patents, city_crosswalk,
#                           on = ['city', 'state'],
#                           how = 'left')

# patents_merged
# #now everywhere has a city/state  
#patents_merged.query("county.isnull() & GEOID.isnull()", engine = 'python').groupby('state').size().sort_values(ascending = False)

In [61]:
#didn't work for some becasue not in US, and/or missing city, etc
#patents_merged.query("county.isnull() & city.isnull() & GEOID.isnull()", engine = 'python')#.groupby('state').size().sort_values(ascending = False)
#patents_merged.query("county.isnull() & GEOID.isnull()", engine = 'python')#.groupby('state').size().sort_values(ascending = False)

In [62]:
#keep places that have county or GEOID
# patents_merged = patents_merged[~(patents_merged["county"].isnull() & patents_merged["GEOID"].isnull())]
# patents_merged

# Get county state fips crosswalk ready for the bulk of patents with county

other data only has state abreeviations, also need state name to match with later dataset

In [42]:
# state abbreviation to state name

states_cross = pd.read_csv("stateabbr_statename.csv")
states_cross.State =  states_cross.State.str.strip().str.upper()
states_cross.head()


,State,Abbrev,Code
0,ALABAMA,Ala.,AL
1,ALASKA,Alaska,AK
2,ARIZONA,Ariz.,AZ
3,ARKANSAS,Ark.,AR
4,CALIFORNIA,Calif.,CA


## territories were included in the data -> only interested in US so got rid of the other places

- examples: Puerto Rico, Guam, etc 

In [64]:
#merge state cross to patents 
# test_merge = pd.merge(patents_merged, states_cross,
#                       left_on = 'state',
#                       right_on = 'Code',
#                       how = 'outer',
#                       indicator = 'matched',
#                       validate = 'many_to_one')

# #some stuff that's not a state
# test_merge.query("matched != 'both'").groupby('state').size()

In [65]:
# patents_merged = pd.merge(patents_merged, states_cross,
#                       left_on = 'state',
#                       right_on = 'Code',
#                       how = 'left')
# patents_merged

In [66]:
# patents_merged.State = patents_merged.State.str.strip().str.upper()
# patents_merged.county = patents_merged.county.str.strip().str.upper()
# patents_merged

# Need to pull out placed that don't have a county listed but must have city to do so -> done above
- patents with no county already match 
- need join the rest

- They can NOT join on county/ state like others
- will need to join on GEOID separately 

In [67]:
#pull out all places with no county 

# patents_no_county = patents_merged.query("county.isnull()", engine = 'python')
# #merge to crosswalk separtely using GeoID
# patents_no_county

## Get the county cross walk ready

In [43]:
#load county crosswalk
county_cross = pd.read_csv('crosswalk_county_name_to_county_fips.csv', dtype = {'GEOID' : 'str'})
county_cross.state_name = county_cross.state_name.str.strip().str.upper()
county_cross.county_name = county_cross.county_name.str.strip().str.upper()
county_cross

,Unnamed: 0,STATEFP,COUNTYFP,GEOID,county_name,state_name
0,0,31,39,31039,CUMING,NEBRASKA
1,1,53,69,53069,WAHKIAKUM,WASHINGTON
2,2,35,11,35011,DE BACA,NEW MEXICO
3,3,31,109,31109,LANCASTER,NEBRASKA
4,4,31,129,31129,NUCKOLLS,NEBRASKA
...,...,...,...,...,...,...
3223,3229,13,123,13123,GILMER,GEORGIA
3224,3230,27,135,27135,ROSEAU,MINNESOTA
3225,3231,28,89,28089,MADISON,MISSISSIPPI
3226,3232,48,227,48227,HOWARD,TEXAS


In [44]:
county_cross.query("GEOID == '24510'")

,Unnamed: 0,STATEFP,COUNTYFP,GEOID,county_name,state_name


In [70]:
#county_cross.query("county_name == 'ST. LOUIS'") #& state_name == 'VIRGINIA'")

## Duplicates in crosswalk

some cities are independent of the county -> dropping the city geoid's
drop:
- BALTIMORE, Marylan with GEOID 24510
- FRANKLIN, VIRGINIA 51620
- 51770 ROANOKE, VIRGINIA
- 51600 FAIRFAX, VIRGINIA
- 51760 RICHMOND, VIRGINIA
- 29510 ST. LOUIS, MISSOUR

In [71]:
drop_list = ['24510', '51620', '51770', '51600', '51760','29510'] 


In [72]:
county_cross = county_cross.query("GEOID != @drop_list")
county_cross

,Unnamed: 0,STATEFP,COUNTYFP,GEOID,county_name,state_name
0,0,31,39,31039,CUMING,NEBRASKA
1,1,53,69,53069,WAHKIAKUM,WASHINGTON
2,2,35,11,35011,DE BACA,NEW MEXICO
3,3,31,109,31109,LANCASTER,NEBRASKA
4,4,31,129,31129,NUCKOLLS,NEBRASKA
...,...,...,...,...,...,...
3223,3229,13,123,13123,GILMER,GEORGIA
3224,3230,27,135,27135,ROSEAU,MINNESOTA
3225,3231,28,89,28089,MADISON,MISSISSIPPI
3226,3232,48,227,48227,HOWARD,TEXAS


In [73]:
#make sure GEOID is a string so don't lose leading zeros 
county_cross = county_cross.assign(GEOID = county_cross.GEOID.astype('str'))

#check that it worked 
county_cross.dtypes

#save without the duplicates for independent cities vs counties
#county_cross.to_csv("crosswalk_county_name_to_county_fips.csv")

Unnamed: 0      int64
STATEFP         int64
COUNTYFP        int64
GEOID          object
county_name    object
state_name     object
dtype: object

In [74]:
# practice merge for the nocounty on GEOID
# test_merge = pd.merge(county_cross, states_cross,
#                        how = 'outer',
#                        left_on = 'state_name',
#                        right_on = 'State',
#                        indicator = 'matched',
#                        validate = 'many_to_one')

# #don;t want these becasue not including trerritories 
# test_merge.query("matched == 'left_only'").groupby('state_name').size()

In [45]:
county_state_crosswalk = pd.merge(county_cross, states_cross,
                        how = 'inner',
                        left_on = 'state_name',
                        right_on = 'State')

county_state_crosswalk = county_state_crosswalk[['GEOID', 'county_name','State', 'Code']]
county_state_crosswalk.head()

,GEOID,county_name,State,Code
0,31039,CUMING,NEBRASKA,NE
1,31109,LANCASTER,NEBRASKA,NE
2,31129,NUCKOLLS,NEBRASKA,NE
3,31101,KEITH,NEBRASKA,NE
4,31137,PHELPS,NEBRASKA,NE


In [76]:
#county_state_crosswalk.to_csv("county_state_crosswalk.csv", index = False)

## Now join patent data with county and state to get geoid

In [46]:
patents_with_county.columns

Index(['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors'],
      dtype='object')

In [47]:
#remove the (CITY) at the end of some counties
import re
regex_pat = re.compile(r'\(CITY\)', flags=re.IGNORECASE)
patents_with_county = patents_with_county.assign(county = patents_with_county.county.str.replace(regex_pat, '', regex = True))

patents_with_county.query("county == 'FALLS CHURCH (CITY)'")

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors


In [48]:
patents_with_county = patents_with_county.assign(county = patents_with_county.county.str.strip().str.upper())

In [80]:
#test
# patents_with_county_geoid = pd.merge(patents_with_county, county_state_crosswalk,
#                                      how = 'outer',
#                                      left_on = ['county', 'state'],
#                                      right_on = ['county_name', 'Code'],
#                                      indicator = 'matched',
#                                      validate = 'many_to_one')

# patents_with_county_geoid.query("matched == 'left_only'").state.value_counts()
                                     

In [81]:
#patents_with_county_geoid.query("matched == 'left_only' & state == 'VA'").county.unique()

In [49]:
patents_with_county_geoid = pd.merge(patents_with_county, county_state_crosswalk,
                                     how = 'inner',
                                     left_on = ['county', 'state'],
                                     right_on = ['county_name', 'Code'])
patents_with_county_geoid

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,GEOID,county_name,State,Code
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,WASHINGTON,A,2,2,2,0,41067,WASHINGTON,OREGON,OR
1,7472496,"NIKE, INC.",2009,2005,11055158,US,BEAVERTON,OR,WASHINGTON,A,2,2,1,1,41067,WASHINGTON,OREGON,OR
2,7472634,"SD3, LLC",2009,2004,10923290,US,TUALATIN,OR,WASHINGTON,B,3,3,3,0,41067,WASHINGTON,OREGON,OR
3,7474419,FEI COMPANY,2009,2006,11590583,US,HILLSBORO,OR,WASHINGTON,G,6,6,5,1,41067,WASHINGTON,OREGON,OR
4,7474972,"TEKTRONIX, INC.",2009,2007,11690797,US,BEAVERTON,OR,WASHINGTON,G,3,3,3,0,41067,WASHINGTON,OREGON,OR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432394,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,WEBSTER CITY,IA,HAMILTON,NaN,1,1,1,0,19079,HAMILTON,IOWA,IA
1432395,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,CRAWFORD,GA,OGLETHORPE,NaN,4,4,3,1,13221,OGLETHORPE,GEORGIA,GA
1432396,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,MINNEAPOLIS,KS,OTTAWA,NaN,2,2,2,0,20143,OTTAWA,KANSAS,KS
1432397,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,FAULKTON,SD,FAULK,NaN,1,1,1,0,46049,FAULK,SOUTH DAKOTA,SD


In [50]:
#all have geoids now
patents_with_county_geoid.GEOID.isnull().sum()

0

In [84]:
#merge no county patents with county codes on GEOID

# patents_no_county_merge = pd.merge(patents_no_county, county_cross,
#                                    how = 'inner',
#                                    on = 'GEOID')
# patents_no_county_merge

## working through the merge

- commented out stuff is from before I removed the patents with no county info

In [85]:
# test_merge = pd.merge(patents_merged, county_cross,
#                       how = 'outer',
#                       left_on = ['county', 'State'],
#                       right_on = ['county_name', 'state_name'],
#                       indicator = 'matched',
#                       validate = 'many_to_one')


In [86]:
#test_merge.query("matched == 'left_only'")

In [87]:
# play = pd.DataFrame(test_merge.query("matched == 'left_only'").groupby(['city', 'state']).size())
# #.query("0 > 1000")#.sort_values(by = 0, ascending = False)
# play = play.rename({0: 'number'}, axis = 1)
# play

In [88]:
#play.query("number > 100").sort_values(by= 'number', ascending = False)

## this is the merge for patents WITH counties on county and state


In [89]:
# patents_merged = pd.merge(patents_merged, county_cross,
#                       how = 'inner',
#                       left_on = ['county', 'State'],
#                       right_on = ['county_name', 'state_name'])
# patents_merged

In [90]:
# working here 
#common = pd.merge(patents_no_county_merge, patents_merged,
#                  on=["patent_number"])
#common
#result = dataframe1[~dataframe1.column1.isin(common.column1)]

# going to concat bu need to be sure same columns first

In [91]:
patents_NO_county_geoid.columns

Index(['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors', 'pv_city', 'GEOID', 'size'],
      dtype='object')

In [51]:
patents_NO_county_geoid.drop(['pv_city', 'size'], axis = 1, inplace = True)
patents_NO_county_geoid

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,GEOID
0,7472531,"FLEECEWORKS, INC.",2009,2005,11218805,US,CITY OF INDUSTRY,CA,NaN,B,1,1,0,1,06037
1,7487807,"LINCOLN GLOBAL, INC.",2009,2006,11497711,US,CITY OF INDUSTRY,CA,NaN,B,2,2,2,0,06037
2,7491910,"LINCOLN GLOBAL, INC.",2009,2006,11387025,US,CITY OF INDUSTRY,CA,NaN,B,3,3,2,1,06037
3,7495193,"LINCOLN GLOBAL, INC.",2009,2005,11081059,US,CITY OF INDUSTRY,CA,NaN,B,3,3,3,0,06037
4,7495766,"LINCCLN GLOBAL, INC.",2009,2006,11472696,US,CITY OF INDUSTRY,CA,NaN,G,2,2,2,0,06037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96599,D865380,"NOBLE HOUSE HOME FURNISHING, LLC",2019,2017,29621633,US,CHATS WORTH,CA,NaN,NaN,2,2,2,0,06037
96600,D867125,"INNO-PAK, LLC",2019,2017,29604217,US,DELWARE,OH,NaN,NaN,2,2,2,0,39041
96601,D869289,"INNO-PAK, LLC",2019,2018,29666033,US,DELWARE,OH,NaN,NaN,2,2,2,0,39041
96602,D867753,"GIFFIN, INC.",2019,2018,29659105,US,LUMMI ISLAND,WA,NaN,NaN,1,1,1,0,53073


In [52]:
patents_with_county_geoid.columns

Index(['patent_number', 'assignee', 'grant_year', 'application_year',
       'application_number', 'country', 'city', 'state', 'county',
       'ipc_section', 'team_size', 'inventors', 'men_inventors',
       'women_inventors', 'GEOID', 'county_name', 'State', 'Code'],
      dtype='object')

In [53]:
patents_with_county_geoid.drop(['county_name', 'State', 'Code'], axis = 1, inplace = True)
patents_with_county_geoid

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,GEOID
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,WASHINGTON,A,2,2,2,0,41067
1,7472496,"NIKE, INC.",2009,2005,11055158,US,BEAVERTON,OR,WASHINGTON,A,2,2,1,1,41067
2,7472634,"SD3, LLC",2009,2004,10923290,US,TUALATIN,OR,WASHINGTON,B,3,3,3,0,41067
3,7474419,FEI COMPANY,2009,2006,11590583,US,HILLSBORO,OR,WASHINGTON,G,6,6,5,1,41067
4,7474972,"TEKTRONIX, INC.",2009,2007,11690797,US,BEAVERTON,OR,WASHINGTON,G,3,3,3,0,41067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432394,D858729,"PELLET FIRE PIT, LLC",2019,2017,29592412,US,WEBSTER CITY,IA,HAMILTON,NaN,1,1,1,0,19079
1432395,D858794,"J & J PORTABLE SANITATION PRODUCTS, LLC",2019,2018,29639264,US,CRAWFORD,GA,OGLETHORPE,NaN,4,4,3,1,13221
1432396,D861112,2 LOTT'S L.L.C.,2019,2017,29598211,US,MINNEAPOLIS,KS,OTTAWA,NaN,2,2,2,0,20143
1432397,D868611,WALKABOUT DEVELOPMENTS LLC,2019,2018,29656834,US,FAULKTON,SD,FAULK,NaN,1,1,1,0,46049


In [54]:
granted_patents = pd.concat([patents_with_county_geoid, patents_NO_county_geoid], ignore_index=True)
sum(granted_patents.duplicated())

0

In [55]:
granted_patents

,patent_number,assignee,grant_year,application_year,application_number,country,city,state,county,ipc_section,team_size,inventors,men_inventors,women_inventors,GEOID
0,7472428,"NIKE, INC.",2009,2004,10814594,US,BEAVERTON,OR,WASHINGTON,A,2,2,2,0,41067
1,7472496,"NIKE, INC.",2009,2005,11055158,US,BEAVERTON,OR,WASHINGTON,A,2,2,1,1,41067
2,7472634,"SD3, LLC",2009,2004,10923290,US,TUALATIN,OR,WASHINGTON,B,3,3,3,0,41067
3,7474419,FEI COMPANY,2009,2006,11590583,US,HILLSBORO,OR,WASHINGTON,G,6,6,5,1,41067
4,7474972,"TEKTRONIX, INC.",2009,2007,11690797,US,BEAVERTON,OR,WASHINGTON,G,3,3,3,0,41067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528998,D865380,"NOBLE HOUSE HOME FURNISHING, LLC",2019,2017,29621633,US,CHATS WORTH,CA,NaN,NaN,2,2,2,0,06037
1528999,D867125,"INNO-PAK, LLC",2019,2017,29604217,US,DELWARE,OH,NaN,NaN,2,2,2,0,39041
1529000,D869289,"INNO-PAK, LLC",2019,2018,29666033,US,DELWARE,OH,NaN,NaN,2,2,2,0,39041
1529001,D867753,"GIFFIN, INC.",2019,2018,29659105,US,LUMMI ISLAND,WA,NaN,NaN,1,1,1,0,53073


In [56]:
#save it
granted_patents.to_csv('granted_patents.csv', index = False)

In [112]:
granted_patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529003 entries, 0 to 1529002
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   patent_number       1529003 non-null  object
 1   assignee            1529003 non-null  object
 2   grant_year          1529003 non-null  int64 
 3   application_year    1529003 non-null  int64 
 4   application_number  1529003 non-null  int64 
 5   country             1529003 non-null  object
 6   city                1529003 non-null  object
 7   state               1529003 non-null  object
 8   county              1432399 non-null  object
 9   ipc_section         1408199 non-null  object
 10  team_size           1529003 non-null  int64 
 11  inventors           1529003 non-null  int64 
 12  men_inventors       1529003 non-null  int64 
 13  women_inventors     1529003 non-null  int64 
 14  GEOID               1529003 non-null  object
dtypes: int64(7), object(8)
memory us